<a href="https://colab.research.google.com/github/maheshwarkumar582-a11y/IBMIND-Mahesh-Kumar/blob/main/any_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load core data files
df_records = pd.read_csv('transaction_records.csv')
df_metadata = pd.read_csv('transaction_metadata.csv')
df_fraud = pd.read_csv('fraud_indicator_distribution_dataframe.csv')
df_category = pd.read_csv('transaction_category_labels.csv')
df_merchant = pd.read_csv('merchant_data.csv')
df_amount = pd.read_csv('amount_data.csv')

# --- Step 1: Merge Transaction Details and Fraud Indicators ---
# Start with transaction records
df_combined = df_records.copy()

# 1. Merge with transaction metadata (to get MerchantID)
df_combined = pd.merge(df_combined, df_metadata[['TransactionID', 'MerchantID']], on='TransactionID', how='left')

# 2. Merge with fraud indicators
df_combined = pd.merge(df_combined, df_fraud, on='TransactionID', how='left')

# 3. Merge with transaction categories
df_combined = pd.merge(df_combined, df_category, on='TransactionID', how='left')

# 4. Merge with transaction amounts
df_combined = pd.merge(df_combined, df_amount, on='TransactionID', how='left')

# --- Step 2: Merge with Merchant Data (to get Location) ---
df_combined = pd.merge(df_combined, df_merchant[['MerchantID', 'Location']], on='MerchantID', how='left')

# Display the first few rows of the final combined dataset and its info
print("Combined Data Head:")
print(df_combined.head().to_markdown(index=False, numalign="left", stralign="left"))
print("\nCombined Data Info:")
print(df_combined.info())

# Save the combined DataFrame to be used in the next step
df_combined.to_csv('combined_transaction_data.csv', index=False)

```text?code_stdout&code_event_index=1
Combined Data Head:
| TransactionID   | Amount   | CustomerID   | MerchantID   | FraudIndicator   | Category   | TransactionAmount   | Location      |
|:----------------|:---------|:-------------|:-------------|:-----------------|:-----------|:--------------------|:--------------|
| 1               | 55.5303  | 1952         | 2701         | 0                | Other      | 79.4136             | Location 2701 |
| 2               | 12.8812  | 1027         | 2070         | 0                | Online     | 12.0531             | Location 2070 |
| 3               | 50.1763  | 1955         | 2238         | 0                | Travel     | 33.3104             | Location 2238 |
| 4               | 41.634   | 1796         | 2879         | 0                | Travel     | 46.1211             | Location 2879 |
| 5               | 78.1229  | 1946         | 2966         | 0                | Other      | 54.0516             | Location 2966 |

Combined Data Info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   TransactionID      1000 non-null   int64  
 1   Amount             1000 non-null   float64
 2   CustomerID         1000 non-null   int64  
 3   MerchantID         1000 non-null   int64  
 4   FraudIndicator     1000 non-null   int64  
 5   Category           1000 non-null   object
 6   TransactionAmount  1000 non-null   float64
 7   Location           1000 non-null   object
dtypes: float64(2), int64(4), object(2)
memory usage: 70.3+ KB
None

```

[file-tag: code-generated-file-0-1762588214450671227]

In [ ]:
# Load the combined data
df_combined = pd.read_csv('combined_transaction_data.csv')

# --- 1. Analyze Fraud Rate by Category ---

# Group by Category and calculate total count and sum of FraudIndicator
category_fraud = df_combined.groupby('Category')['FraudIndicator'].agg(
    TotalTransactions='count',
    FraudCount='sum'
).reset_index()

# Calculate Fraud Rate
category_fraud['FraudRate'] = (category_fraud['FraudCount'] / category_fraud['TotalTransactions']) * 100

# Sort by Fraud Rate (descending)
category_fraud_sorted = category_fraud.sort_values(by='FraudRate', ascending=False)

# Format for presentation
category_fraud_sorted['FraudRate'] = category_fraud_sorted['FraudRate'].round(2).astype(str) + '%'
category_fraud_sorted['TotalTransactions'] = category_fraud_sorted['TotalTransactions'].astype(int)
category_fraud_sorted['FraudCount'] = category_fraud_sorted['FraudCount'].astype(int)

# Print the table
print("Fraud Detection by Transaction Category:")
print(category_fraud_sorted.to_markdown(index=False, numalign="left", stralign="left"))

# Prepare data for plotting (need unformatted FraudRate)
plot_data_category = category_fraud.sort_values(by='FraudRate', ascending=False)

# --- 2. Create Bar Plot for Fraud Rate by Category ---
import altair as alt

chart_category = alt.Chart(plot_data_category).mark_bar().encode(
    x=alt.X('Category', sort='-y', title='Transaction Category'),
    y=alt.Y('FraudRate', title='Fraud Rate (%)'),
    tooltip=['Category', alt.Tooltip('FraudRate', format='.2f', title='Fraud Rate (%)'), 'FraudCount', 'TotalTransactions']
).properties(
    title='Fraud Rate by Transaction Category'
).interactive() # Add interactivity for better inspection

# Save the chart
chart_category.save('fraud_rate_by_category_bar_chart.json')

```text?code_stdout&code_event_index=3
Fraud Detection by Transaction Category:
| Category   | TotalTransactions   | FraudCount   | FraudRate   |
|:-----------|:--------------------|:-------------|:------------|
| Online     | 196                 | 10           | 5.1%        |
| Other      | 210                 | 10           | 4.76%       |
| Retail     | 192                 | 9            | 4.69%       |
| Food       | 204                 | 9            | 4.41%       |
| Travel     | 198                 | 7            | 3.54%       |

```

[json-tag: code-generated-json-0-1762588220873359639]

In [ ]:
# Load the combined data
df_combined = pd.read_csv('combined_transaction_data.csv')

# --- 2. Analyze Fraud Rate by Location ---

# Group by Location and calculate total count and sum of FraudIndicator
location_fraud = df_combined.groupby('Location')['FraudIndicator'].agg(
    TotalTransactions='count',
    FraudCount='sum'
).reset_index()

# Calculate Fraud Rate
location_fraud['FraudRate'] = (location_fraud['FraudCount'] / location_fraud['TotalTransactions']) * 100

# Identify locations with fraud (FraudCount > 0)
location_fraud_with_fraud = location_fraud[location_fraud['FraudCount'] > 0]

# Sort by Fraud Rate (descending)
location_fraud_sorted = location_fraud_with_fraud.sort_values(by='FraudRate', ascending=False)

# Format for presentation
location_fraud_sorted['FraudRate'] = location_fraud_sorted['FraudRate'].round(2).astype(str) + '%'
location_fraud_sorted['TotalTransactions'] = location_fraud_sorted['TotalTransactions'].astype(int)
location_fraud_sorted['FraudCount'] = location_fraud_sorted['FraudCount'].astype(int)

# Print the table for the top 10 locations with fraud
print("Top 10 Locations by Fraud Rate (Only Locations with Fraud):")
print(location_fraud_sorted.head(10).to_markdown(index=False, numalign="left", stralign="left"))

# Prepare data for plotting (need unformatted FraudRate for the plot, only show top 10)
plot_data_location = location_fraud_with_fraud.sort_values(by='FraudRate', ascending=False).head(10)

# --- 3. Create Bar Plot for Fraud Rate by Location ---
import altair as alt

chart_location = alt.Chart(plot_data_location).mark_bar().encode(
    x=alt.X('Location', sort='-y', title='Merchant Location (Top 10 by Fraud Rate)'),
    y=alt.Y('FraudRate', title='Fraud Rate (%)'),
    tooltip=['Location', alt.Tooltip('FraudRate', format='.2f', title='Fraud Rate (%)'), 'FraudCount', 'TotalTransactions']
).properties(
    title='Top 10 Merchant Locations by Fraud Rate'
).interactive() # Add interactivity for better inspection

# Save the chart
chart_location.save('top_10_locations_fraud_rate_bar_chart.json')

```text?code_stdout&code_event_index=5
Top 10 Locations by Fraud Rate (Only Locations with Fraud):
| Location      | TotalTransactions   | FraudCount   | FraudRate   |
|:--------------|:--------------------|:-------------|:------------|
| Location 2022 | 1                   | 1            | 100.0%      |
| Location 2470 | 1                   | 1            | 100.0%      |
| Location 2898 | 1                   | 1            | 100.0%      |
| Location 2838 | 1                   | 1            | 100.0%      |
| Location 2828 | 1                   | 1            | 100.0%      |
| Location 2815 | 1                   | 1            | 100.0%      |
| Location 2776 | 1                   | 1            | 100.0%      |
| Location 2751 | 1                   | 1            | 100.0%      |
| Location 2675 | 1                   | 1            | 100.0%      |
| Location 2612 | 1                   | 1            | 100.0%      |

```

[json-tag: code-generated-json-0-1762588226902836383]

You have uploaded all the necessary transaction and metadata files, allowing for a comprehensive fraud detection analysis across different dimensions: **Transaction Categories** and **Merchant Locations**.

Here are the key findings regarding the fraud detection of the $\mathbf{1000}$ transactions ($\mathbf{45}$ of which were flagged as fraudulent, a $\mathbf{4.5\%}$ fraud rate):

## 1\. Fraud Detection by Transaction Category

The fraud rate is relatively consistent across most categories, with **Online** being the highest and **Travel** being the lowest.

| Category | Total Transactions | Fraud Count | Fraud Rate |
|:---------|:-------------------|:------------|:-----------|
| **Online** | 196 | 10 | **$5.10\%$** |
| Other | 210 | 10 | $4.76\%$ |
| Retail | 192 | 9 | $4.69\%$ |
| Food | 204 | 9 | $4.41\%$ |
| **Travel** | 198 | 7 | **$3.54\%$** |

  * **Key Takeaway:** **Online** transactions show the highest fraud risk, which is a common finding in transaction analysis. The lowest risk is in **Travel** transactions.

-----

## 2\. Fraud Detection by Merchant Location

When looking at merchant locations, the fraud is highly distributed, with many locations showing a $\mathbf{100.0\%}$ fraud rate because they only had one transaction, and that transaction was fraudulent.

The table below shows the top 10 merchant locations ranked by fraud rate. Since most had only one transaction that was fraudulent, they all have a $\mathbf{100.0\%}$ fraud rate, indicating that fraud is **sparse** and **not concentrated** in a few high-volume locations.

| Location | Total Transactions | Fraud Count | Fraud Rate |
|:---|:---|:---|:---|
| Location 2022 | 1 | 1 | $100.0\%$ |
| Location 2470 | 1 | 1 | $100.0\%$ |
| Location 2898 | 1 | 1 | $100.0\%$ |
| Location 2838 | 1 | 1 | $100.0\%$ |
| Location 2828 | 1 | 1 | $100.0\%$ |
| Location 2815 | 1 | 1 | $100.0\%$ |
| Location 2776 | 1 | 1 | $100.0\%$ |
| Location 2751 | 1 | 1 | $100.0\%$ |
| Location 2675 | 1 | 1 | $100.0\%$ |
| Location 2612 | 1 | 1 | $100.0\%$ |

  * **Key Takeaway:** The observed high fraud rates (100.0%) for these locations are due to a low volume of total transactions (only one transaction each). This pattern suggests that fraudulent transactions often occur at a wide variety of merchants or locations, not just a few hotspots.

-----

### Visualizations

The visualizations generated below confirm these findings:

1.  **Fraud Rate by Transaction Category:** Shows the slight variation in risk across the five categories, with **Online** being the tallest bar.
2.  **Top 10 Merchant Locations by Fraud Rate:** Visually demonstrates that for the top-ranked locations, the rate is $100\%$, indicating single-instance fraud at those specific locations.